<a href="https://colab.research.google.com/github/mega-317/medical-specialty-prediction/blob/main/specialty_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import numpy as np

# 1. KM-BERT 불러오기
model_name = "madatnlp/km-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name)

# 2. 진료과 정의
specialties = ["내과", "신경과", "정신과", "소아청소년과", "정형외과"]
num_classes = len(specialties)

# 3. BERT + 분류기 구성
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=256, num_classes=num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.fc1 = nn.Linear(bert.config.hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_token = outputs.last_hidden_state[:, 0, :]  # [CLS] 토큰 사용
        x = self.fc1(cls_token)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 4. 학습 데이터 준비
texts = [
    "열이 나고 기침이 심해요",        # 내과
    "두통이 계속되고 어지러워요",      # 신경과
    "최근 불안하고 우울감이 심해요",    # 정신과
    "아이 열이 계속 나요",             # 소아청소년과
    "팔이 부러졌어요",                # 정형외과
]
labels = [0, 1, 2, 3, 4]  # specialties 인덱스

# 5. 토크나이징
encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")

# 6. 데이터셋 클래스 정의
class SymptomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = SymptomDataset(encodings, labels)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

# 7. 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERTClassifier(bert_model)
model.to(device)

# 옵티마이저, 손실 함수 설정
optimizer = optim.Adam(model.parameters(), lr=3e-5)
criterion = nn.CrossEntropyLoss()

# 8. 모델 학습
model.train()
epochs = 10

for epoch in range(epochs):
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")

# 9. 예측 함수
def predict_specialty(sentence):
    model.eval()
    inputs = tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.softmax(outputs, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
    return specialties[pred_class]

# 10. 사용자 입력 테스트
user_sentence = "자고 일어날 때마다 자꾸 머리가 아파요"
predicted_specialty = predict_specialty(user_sentence)
print(f"예측된 진료과: {predicted_specialty}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Epoch 1 | Loss: 5.0252
Epoch 2 | Loss: 4.8914
Epoch 3 | Loss: 4.4426
Epoch 4 | Loss: 4.0846
Epoch 5 | Loss: 3.8952
Epoch 6 | Loss: 4.4047
Epoch 7 | Loss: 4.6841
Epoch 8 | Loss: 3.8296
Epoch 9 | Loss: 3.3821
Epoch 10 | Loss: 3.3543
예측된 진료과: 신경과
